# MODELO 3 - ....

# POR ACABAR !!!!!!!!!!

#### use faa03 kernel

In [9]:
import pandas as pd

# Load the data

In [10]:
df_train = pd.read_csv("data/data_train.csv")
df_test = pd.read_csv("data/data_test.csv")

In [11]:
from surprise import Dataset, Reader
from surprise import SVD
from surprise import accuracy
import pandas as pd

# Load your ratings data
ratings_df = df_train

# Define the format for Surprise (user, item, rating)
reader = Reader(rating_scale=(1, 5))  # Adjust rating scale based on your data
data = Dataset.load_from_df(ratings_df[['userId', 'movieId', 'rating']], reader)

# Split the data into trainset and testset (optional)
trainset = data.build_full_trainset()  # Use the full data for prediction
testset = trainset.build_testset()  # Use the full testset (or a specific subset)

# Initialize the SVD algorithm
algo = SVD()

# Train the model
algo.fit(trainset)

# Predict ratings for all combinations of users and movies in the testset
predictions = algo.test(testset)

# Optional: Evaluate the model (e.g., RMSE)
rmse = accuracy.rmse(predictions)
print(f'Root Mean Squared Error: {rmse}')

# Create a dictionary of predictions
all_predictions = {}
for prediction in predictions:
    user = prediction.uid
    item = prediction.iid
    predicted_rating = prediction.est
    if user not in all_predictions:
        all_predictions[user] = {}
    all_predictions[user][item] = predicted_rating

# Now `all_predictions` contains all predicted ratings


RMSE: 0.6391
Root Mean Squared Error: 0.639061006791933


In [14]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import cross_validate

# Load your ratings data
ratings_df = pd.read_csv('data/ratings.csv')  # Ensure this is the correct path

# Define the format for Surprise (user, item, rating)
reader = Reader(rating_scale=(1, 5))  # Adjust rating scale based on your data
data = Dataset.load_from_df(ratings_df[['userId', 'movieId', 'rating']], reader)

# Initialize the SVD algorithm
algo = SVD()

# Perform 8-fold cross-validation
cv_results = cross_validate(algo, data, cv=8, measures=['RMSE', 'MAE'], verbose=True)

# Average results over the 8 folds
print(f"Average RMSE: {cv_results['test_rmse'].mean():.4f}")
print(f"Average MAE: {cv_results['test_mae'].mean():.4f}")

Evaluating RMSE, MAE of algorithm SVD on 8 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Mean    Std     
RMSE (testset)    0.8620  0.8807  0.8678  0.8611  0.8636  0.8701  0.8727  0.8659  0.8680  0.0061  
MAE (testset)     0.6650  0.6779  0.6653  0.6593  0.6619  0.6650  0.6669  0.6676  0.6661  0.0051  
Fit time          0.45    0.44    0.43    0.42    0.42    0.42    0.42    0.42    0.43    0.01    
Test time         0.03    0.10    0.03    0.03    0.03    0.08    0.03    0.03    0.04    0.03    
Average RMSE: 0.8680
Average MAE: 0.6661


In [17]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import GridSearchCV
import pandas as pd

# Load the data
ratings_df = df_train
reader = Reader(rating_scale=(1, 5))  # Adjust rating scale if needed
data = Dataset.load_from_df(ratings_df[['userId', 'movieId', 'rating']], reader)

# Define hyperparameter grid for SVD
param_grid = {
    'n_factors': [50, 100, 150],        # Number of latent factors
    'lr_all': [0.002, 0.005, 0.01],    # Learning rate
    'reg_all': [0.02, 0.1, 0.2]        # Regularization term
}

# Perform 8-fold cross-validation for hyperparameter tuning
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=8)
gs.fit(data)

# Get the best parameters and the corresponding RMSE
print(f"Best RMSE: {gs.best_score['rmse']:.4f}")
print("Best Hyperparameters:", gs.best_params['rmse'])

# Train the best model on the entire dataset
best_algo = gs.best_estimator['rmse']
trainset = data.build_full_trainset()
best_algo.fit(trainset)

# Predict all ratings
all_user_ids = trainset.all_users()  # Internal user IDs
all_item_ids = trainset.all_items()  # Internal item IDs

# Generate predictions for all user-item pairs
all_predictions = []
for user in all_user_ids:
    for item in all_item_ids:
        all_predictions.append(best_algo.predict(user, item))

# Convert predictions to a more readable format (optional)
predictions_df = pd.DataFrame([{
    'userId': pred.uid,
    'movieId': pred.iid,
    'predicted_rating': pred.est
} for pred in all_predictions])

# Display sample predictions
print(predictions_df.head())


Best RMSE: 0.8643
Best Hyperparameters: {'n_factors': 100, 'lr_all': 0.01, 'reg_all': 0.1}
   userId  movieId  predicted_rating
0       0        0          3.502572
1       0        1          3.856956
2       0        2          3.470802
3       0        3          3.246449
4       0        4          2.794946


In [ ]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split, GridSearchCV
from surprise import accuracy
import pandas as pd

# Load the data
ratings_df = pd.read_csv('data/ratings.csv')  # Adjust the path as necessary
reader = Reader(rating_scale=(1, 5))  # Adjust rating scale if needed
data = Dataset.load_from_df(ratings_df[['userId', 'movieId', 'rating']], reader)

# Step 1: Perform grid search on the original Dataset object
param_grid = {
    'n_factors': [50, 100, 150],        # Number of latent factors
    'lr_all': [0.002, 0.005, 0.01],    # Learning rate
    'reg_all': [0.02, 0.1, 0.2]        # Regularization term
}

# Use the original Dataset for grid search
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=8)
gs.fit(data)  # Use Dataset, not Trainset

# Step 2: Get the best parameters and retrain the model
print(f"Best RMSE: {gs.best_score['rmse']:.4f}")
print("Best Hyperparameters:", gs.best_params['rmse'])
best_algo = gs.best_estimator['rmse']

# Step 3: Split data into train and test sets
trainset, testset = train_test_split(data, test_size=0.2)  # 80% train, 20% test # MAS ASSIM ELE JA VIU OS DADOS !!!!!

# Step 4: Fit the best model on the training set
best_algo.fit(trainset)

# Step 5: Evaluate on the test set
predictions = best_algo.test(testset)

# Step 6: Compute metrics on the test set
rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)

# Optional: Save predictions to a DataFrame for analysis
predictions_df = pd.DataFrame([{
    'userId': pred.uid,
    'movieId': pred.iid,
    'true_rating': pred.r_ui,
    'predicted_rating': pred.est
} for pred in predictions])

print(predictions_df.head())


Best RMSE: 0.8571
Best Hyperparameters: {'n_factors': 150, 'lr_all': 0.01, 'reg_all': 0.1}
RMSE: 0.8592
MAE:  0.6628
   userId  movieId  true_rating  predicted_rating
0     537   104218          3.5          3.923755
1     393     4022          5.0          3.983535
2     448     5309          2.0          2.644740
3     510     3360          3.5          3.066725
4     223     7541          3.5          3.065923
